In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F



spark = (
        SparkSession
            .builder
            .appName('ResourceAllocationApp')
            .master('local[*]')
            
            #.master('spark://C02D108AMD6W:7077')

            .config('spark.dynamicAllocation.enabled', 'true')

            .config('spark.sql.adaptive.enabled', 'false')
            
            .config('spark.sql.adaptive.skewJoin.enabled', 'false')


            
            #.config('spark.dynamicAllocation.shuffleTracking.enabled', 'true')
#
            #
            #.config('spark.executor.memory', '2g')
            #.config('spark.executor.cores', '2')
#
            #.config('spark.executor.instances', '2')
#
            #.config('spark.dynamicAllocation.minExecutors', '0')
            #.config('spark.dynamicAllocation.maxExecutors', '5')
#
            #.config('spark.dynamicAllocation.schedulerBacklogTimeout', '1s')
            #.config('spark.dynamicAllocation.executorIdleTimeout', '10s')
            #.config('spark.dynamicAllocation.cachedExecutorIdleTimeout', '10s')
            
            .getOrCreate()

    )

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/23 10:50:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
productsDf = (
    spark   
        .range(1, 2000001)
        .select(
            F.col('id').alias('ProductId'),
            F.expr('ROUND(RAND() * 100, 2) AS Price')
        )
)

In [3]:
productsDf.createOrReplaceTempView('Products')

In [4]:
salesDf = (
    spark
        .range(1, 100000001)
        .select(
            F.col('id').alias('SalesId'),
            F.expr(
                """
                    CASE WHEN RAND() < 0.7
                        THEN 1
                    ELSE
                        CAST (RAND() * 2000000 AS INT)
                        END
                """
            ).alias('ProductId'),
            F.expr('CAST(RAND() * 10 AS INTEGER)').alias('QuantitySold'),
            F.expr('DATE_ADD(CURRENT_DATE(), - CAST(RAND() * 365 AS INT))').alias('SalesState')

        )
)



In [5]:
salesDf.createOrReplaceTempView('Sales')

In [ ]:
#spark.sql("""
#    SELECT ProductId, COUNT(*) AS ProductCount
#          FROM Sales
#          Group By ProductId
#          ORDER BY ProductCount DESC
#
#
#""").show()

23/09/23 22:45:32 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 4109688 ms exceeds timeout 120000 ms
23/09/23 22:45:32 WARN SparkContext: Killing executors is not supported by current scheduler.
23/09/23 22:45:34 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at s

In [5]:
yellow_taxi_df = (
    spark.read
    .option('header', 'true')
    .option('inferSchema', 'true')
    .csv('../data/YellowTaxis_202210.csv')
)

yellow_taxi_df_grouped =  (yellow_taxi_df.dropDuplicates()
    .groupBy('PULocationID')
    .agg(sum('total_amount')))

In [7]:
yellow_taxi_df_grouped.show(10)

+------------+------------------+
|PULocationID| sum(total_amount)|
+------------+------------------+
|         148| 795087.6300000009|
|         243|16739.699999999986|
|          31| 935.1500000000001|
|         137| 762800.5900000011|
|          85|3172.5600000000004|
|         251|            521.63|
|          65| 80190.23000000001|
|         255| 51031.78000000001|
|          53|1601.2899999999995|
|         133|3281.4800000000023|
+------------+------------------+
only showing top 10 rows



23/09/18 21:14:45 WARN TransportChannelHandler: Exception in connection from C02D108AMD6W/192.168.81.116:7077
java.io.IOException: Operation timed out
	at java.base/sun.nio.ch.SocketDispatcher.read0(Native Method)
	at java.base/sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:47)
	at java.base/sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:330)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:284)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:259)
	at java.base/sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:417)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:259)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1132)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:357)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:151)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:788)
	at io.netty.channel.nio.NioEventLoop.processSelec